# Minneapolis, MN Simulations
Need to add Scenario field to CSV files.<br>
8760 rows per simulation<br>
49 files 

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 0.82
elec = 0.1314

# Size (in sq ft) of the building to establish EUI
size = 2400

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

---

## Combine v9.5 CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/95/msp/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data95 = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/95/msp/" + file)
    all_data95 = pd.concat([all_data95, df])
    
all_data95.head()

# 429240  rows, 214 columns

,Scenario,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Exterior Horizontal Sky Illuminance [lux](Hourly),Environment:Site Exterior Horizontal Beam Illuminance [lux](Hourly),Environment:Site Exterior Beam Normal Illuminance [lux](Hourly),Environment:Site Sky Diffuse Solar Radiation Luminous Efficacy [lum/W](Hourly),Environment:Site Beam Solar Radiation Luminous Efficacy [lum/W](Hourly),...,WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),LIVING_UNIT1:Zone Operative Temperature [C](Hourly),ATTIC_UNIT1:Zone Operative Temperature [C](Hourly),SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly)
0,1,01/01 01:00:00,-6.6500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.473440,-2.452388,378211.7161,24412005.56,0.0
1,1,01/01 02:00:00,-7.9875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.442552,-4.927196,424942.6963,27428297.48,0.0
2,1,01/01 03:00:00,-7.4250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.423690,-5.761685,436672.6658,28185418.61,0.0
3,1,01/01 04:00:00,-6.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.406847,-5.776410,436173.5662,28153203.79,0.0
4,1,01/01 05:00:00,-6.3250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.393771,-5.509716,446361.0315,28810762.64,0.0


In [5]:
# Get wanted columns
all_data95_1 = all_data95[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

# all_data95_1

In [6]:
# all_data95_1 = all_data95_1.dropna().reset_index(drop=True)

# all_data95_1

# #438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [7]:
# Rename Columns
all_data95_2 = all_data95_1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data95_2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [8]:
# Copy dataframe to modify and leave original df intact
get_annual95 = all_data95_2.copy(deep=True)

#get_annual

In [9]:
# Convert Joules to KBtu in dataframe
get_annual95["FanEnergy(kBtu)"] = get_annual95["FanEnergy[J](Hourly)"] * .00000094781712
get_annual95["HeatingEnergy(kBtu)"] = get_annual95["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual95["CoolingEnergy(kBtu)"] = get_annual95["CoolingEnergy[J](Hourly) "] * .00000094781712

# get_annual95

In [10]:
# Get the annual energy per square foot
# Square foot is defined in the size variable

get_annual95["FanEnergy(kBtu/sf)"] = get_annual95["FanEnergy(kBtu)"] / size
get_annual95["HeatingEnergy(kBtu/sf)"] = get_annual95["HeatingEnergy(kBtu)"] / size
get_annual95["CoolingEnergy(kBtu/sf)"] = get_annual95["CoolingEnergy(kBtu)"] / size

In [11]:
# Drop columns
get_annual95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# get_annual95.head()

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [12]:
# Get annual costs
get_annual95_costs = all_data95_2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual95_costs["FanCost($)"] = (get_annual95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual95_costs["HeatingCost($)"] = ((get_annual95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual95_costs["CoolingCost($)"] = (get_annual95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs95

In [14]:
# Get the annual energy per square foot
# Square foot is defined in the size variable

get_annual95_costs["FanCost($/sf)"] = get_annual95_costs["FanCost($)"] / size
get_annual95_costs["HeatingCost($/sf)"] = get_annual95_costs["HeatingCost($)"] / size
get_annual95_costs["CoolingCost($/sf)"] = get_annual95_costs["CoolingCost($)"] / size

In [15]:
# Drop columns
get_annual95_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# get_annual95_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [16]:
# Combine data for energy and cost
allAnnualData95 = pd.merge(get_annual95, get_annual95_costs)

allAnnualData95.head()

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy(kBtu/sf),HeatingEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),FanCost($),HeatingCost($),CoolingCost($),FanCost($/sf),HeatingCost($/sf),CoolingCost($/sf)
0,1,01/01 01:00:00,0.0,0.358476,23.138117,0.0,0.000149,0.009641,0.0,0.013805,0.189733,0.0,0.000006,0.000079,0.0
1,1,01/01 02:00:00,0.0,0.402768,25.997010,0.0,0.000168,0.010832,0.0,0.015510,0.213175,0.0,0.000006,0.000089,0.0
2,1,01/01 03:00:00,0.0,0.413886,26.714622,0.0,0.000172,0.011131,0.0,0.015939,0.219060,0.0,0.000007,0.000091,0.0
3,1,01/01 04:00:00,0.0,0.413413,26.684089,0.0,0.000172,0.011118,0.0,0.015920,0.218810,0.0,0.000007,0.000091,0.0
4,1,01/01 05:00:00,0.0,0.423069,27.307334,0.0,0.000176,0.011378,0.0,0.016292,0.223920,0.0,0.000007,0.000093,0.0


In [17]:
# Add energy and cost totals
totalEnergy95 = allAnnualData95["FanEnergy(kBtu)"] + allAnnualData95["HeatingEnergy(kBtu)"] + allAnnualData95["CoolingEnergy(kBtu)"]
allAnnualData95["AnnualEnergy(kBtu)"] = totalEnergy95

totalEnergyEUI95 = allAnnualData95["FanEnergy(kBtu/sf)"] + allAnnualData95["HeatingEnergy(kBtu/sf)"] + allAnnualData95["CoolingEnergy(kBtu/sf)"]
allAnnualData95["AnnualEnergy(kBtu/sf)"] = totalEnergyEUI95


totalCost95 = allAnnualData95["FanCost($)"] + allAnnualData95["HeatingCost($)"] + allAnnualData95["CoolingCost($)"]
allAnnualData95["AnnualCost($)"] = totalCost95

totalCostEUI95 = allAnnualData95["FanCost($/sf)"] + allAnnualData95["HeatingCost($/sf)"] + allAnnualData95["CoolingCost($/sf)"]
allAnnualData95["AnnualCost($/sf)"] = totalCostEUI95


allAnnualData95.head()

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy(kBtu/sf),HeatingEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),FanCost($),HeatingCost($),CoolingCost($),FanCost($/sf),HeatingCost($/sf),CoolingCost($/sf),AnnualEnergy(kBtu),AnnualEnergy(kBtu/sf),AnnualCost($),AnnualCost($/sf)
0,1,01/01 01:00:00,0.0,0.358476,23.138117,0.0,0.000149,0.009641,0.0,0.013805,0.189733,0.0,0.000006,0.000079,0.0,23.496592,0.009790,0.203537,0.000085
1,1,01/01 02:00:00,0.0,0.402768,25.997010,0.0,0.000168,0.010832,0.0,0.015510,0.213175,0.0,0.000006,0.000089,0.0,26.399778,0.011000,0.228686,0.000095
2,1,01/01 03:00:00,0.0,0.413886,26.714622,0.0,0.000172,0.011131,0.0,0.015939,0.219060,0.0,0.000007,0.000091,0.0,27.128508,0.011304,0.234998,0.000098
3,1,01/01 04:00:00,0.0,0.413413,26.684089,0.0,0.000172,0.011118,0.0,0.015920,0.218810,0.0,0.000007,0.000091,0.0,27.097501,0.011291,0.234730,0.000098
4,1,01/01 05:00:00,0.0,0.423069,27.307334,0.0,0.000176,0.011378,0.0,0.016292,0.223920,0.0,0.000007,0.000093,0.0,27.730403,0.011554,0.240212,0.000100


In [18]:
#list(allAnnualData95.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [19]:
# Group by scenario
annualData95 = allAnnualData95.groupby(["Scenario"], as_index=True)

#annualData95.sum().round(2)

In [20]:
annualDataFinal95 = annualData95.sum().round(2)

annualDataFinal95.head()

,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy(kBtu/sf),HeatingEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),FanCost($),HeatingCost($),CoolingCost($),FanCost($/sf),HeatingCost($/sf),CoolingCost($/sf),AnnualEnergy(kBtu),AnnualEnergy(kBtu/sf),AnnualCost($),AnnualCost($/sf)
Scenario,,,,,,,,,,,,,,,,,
1,17883055.31,1952.94,99255.59,4463.11,0.81,41.36,1.86,75.21,813.90,171.87,0.03,0.34,0.07,105671.64,44.03,1060.97,0.44
2,17883055.31,1997.12,95717.19,5494.41,0.83,39.88,2.29,76.91,784.88,211.59,0.03,0.33,0.09,103208.72,43.00,1073.38,0.45
3,17883055.31,2048.08,92588.41,6520.17,0.85,38.58,2.72,78.87,759.22,251.09,0.03,0.32,0.10,101156.67,42.15,1089.18,0.45
4,18555070.10,2108.89,89572.92,7619.66,0.88,37.32,3.17,81.21,734.50,293.43,0.03,0.31,0.12,99301.46,41.38,1109.14,0.46
5,19227229.59,2179.21,86664.56,8789.60,0.91,36.11,3.66,83.92,710.65,338.48,0.03,0.30,0.14,97633.38,40.68,1133.05,0.47


In [21]:
# Export grouped file to csv
annualDataFinal95.to_csv("../Regression_Scraper_Output/msp_annual95_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh for costs only, convert to kbtu for data<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [22]:
# # Copy dataframe to modify and leave original df intact
# get_monthly95 = all_data95_2.copy(deep=True)

# #get_monthly95

In [23]:
# # Convert Joules to kWh and therms in new dataframe
# get_monthly95["FanEnergy(kBtu)"] = get_monthly95["FanEnergy[J](Hourly)"] * .00000094781712
# get_monthly95["HeatingEnergy(kBtu)"] = get_monthly95["HeatingEnergy[J](Hourly)"] * .00000094781712
# get_monthly95["CoolingEnergy(kBtu)"] = get_monthly95["CoolingEnergy[J](Hourly) "] * .00000094781712

# #get_monthly95

In [24]:
# # Drop columns
# get_monthly95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# get_monthly95

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [25]:
# # Copy dataframe to modify and leave original df intact
# get_monthly95_costs = all_data95_2.copy(deep=True)

# #get_monthly95

In [26]:
# # Convert Joules to KBtu in dataframe and add cost multiplier
# # Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# # Elec kbtu => Kwh (kbtu * .000000277777778)
# get_monthly95_costs["FanCost($)"] = (get_monthly95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
# get_monthly95_costs["HeatingCost($)"] = ((get_monthly95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
# get_monthly95_costs["CoolingCost($)"] = (get_monthly95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

# #get_monthly95_costs

In [27]:
# # Drop columns
# get_monthly95_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_monthly95_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [28]:
# # Combine data for energy and cost
# allMonthlyData95 = pd.merge(get_monthly95, get_monthly95_costs, how="inner")

# #allMonthlyData95

In [29]:
# # Add energy and cost totals

# totalEnergyMonthly95 = allMonthlyData95["FanEnergy(kBtu)"] + allMonthlyData95["HeatingEnergy(kBtu)"] + allMonthlyData95["CoolingEnergy(kBtu)"]
# totalCostMonthly95 = allMonthlyData95["FanCost($)"] + allMonthlyData95["HeatingCost($)"] + allMonthlyData95["CoolingCost($)"]
# allMonthlyData95["MonthlyEnergy(kBtu)"] = totalEnergyMonthly95
# allMonthlyData95["MonthlyCost($)"] = totalCostMonthly95

# #allMonthlyData95

In [30]:
# allMonthlyData95["Month"] = allMonthlyData95.Date_Time.str.slice(0,3)

# allMonthlyData95

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [31]:
# # Group by scenario and date/time for export to graphing
# MonthlyData95_2 = allMonthlyData95.groupby(["Scenario", "Month"], as_index=True)

# #MonthlyData95_2.sum()

In [32]:
# MonthlyDataFinal95 = MonthlyData95_2.sum()

# MonthlyDataFinal95

In [33]:
# # Drop Illuminance column...don't need the totals
# MonthlyDataFinal95.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [34]:
# # Export grouped file to csv (used before grouped by month)
# MonthlyDataFinal95.round(2).to_csv("../Regression_Scraper_Output/msp_monthly95_data.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [35]:
# # Copy dataframe to modify and leave original df intact
# all_data95_3 = all_data95_1.copy(deep=True)

# #all_data95_3

In [36]:
# # Get wanted columns
# # Include hourly illumance
# all_data95_3 = all_data95_3[["Scenario",
#                "Date/Time",
#                              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
#                              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
#                              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
#                              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
#                       ]]

# #all_data95_3

In [37]:
# # Rename Columns
# get_hourly95 = all_data95_3.rename(columns={"Scenario":"Scenario",
#                               "Date/Time":"Date_Time",
#                              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
#                              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
#                              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
#                              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
#                              })

# #get_hourly95

In [38]:
# # Convert Joules to kBtus in new dataframe
# get_hourly95["FanEnergy(kBtu)"] = get_hourly95["FanEnergy[J](Hourly)"] * .00000094781712
# get_hourly95["HeatingEnergy(kBtu)"] = get_hourly95["HeatingEnergy[J](Hourly)"] * .00000094781712
# get_hourly95["CoolingEnergy(kBtu)"] = get_hourly95["CoolingEnergy[J](Hourly) "] * .00000094781712

# #get_hourly95

In [39]:
# # Drop columns
# get_hourly95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_hourly95

In [40]:
#list(get_hourly95.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [41]:
# # Copy dataframe to modify and leave original df intact
# get_hourly95_costs = all_data95_3.copy(deep=True)

# #get_hourly95_costs

In [42]:
# # Rename Columns
# get_hourly95_costs = get_hourly95_costs.rename(columns={"Scenario":"Scenario",
#                               "Date/Time":"Date_Time",
#                              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
#                              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
#                              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
#                              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
#                              })

# # get_hourly95_costs

In [43]:
# # Convert Joules to KBtu in dataframe and add cost multiplier
# # Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# # Elec kbtu => Kwh (kbtu * .000000277777778)
# get_hourly95_costs["FanCost($)"] = (get_hourly95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
# get_hourly95_costs["HeatingCost($)"] = ((get_hourly95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
# get_hourly95_costs["CoolingCost($)"] = (get_hourly95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

# ##get_hourly95_costs

In [44]:
# # Drop columns
# get_hourly95_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_hourly95_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [45]:
# # Combine data for energy and cost
# allHourlyData95 = pd.merge(get_hourly95, get_hourly95_costs, how="inner")

# #allHourlyData95

In [46]:
# #Extract hour, day, month.
# allHourlyData95["Month"] = allHourlyData95.Date_Time.str.slice(0,3)
# allHourlyData95["Day"] = allHourlyData95.Date_Time.str.slice(4,6)
# allHourlyData95["Hour"] = allHourlyData95["Date_Time"].astype(str).str[8:16]

# allHourlyData95

In [47]:
# # Add energy and cost totals

# totalEnergyhourly95 = allHourlyData95["FanEnergy(kBtu)"] + allHourlyData95["HeatingEnergy(kBtu)"] + allHourlyData95["CoolingEnergy(kBtu)"]
# totalCosthourly95 = allHourlyData95["FanCost($)"] + allHourlyData95["HeatingCost($)"] + allHourlyData95["CoolingCost($)"]
# allHourlyData95["HourlyEnergy(kBtu)"] = totalEnergyhourly95
# allHourlyData95["HourlyCost($)"] = totalCosthourly95

# #allHourlyData95

In [48]:
# # Export grouped file to csv (used before grouped by month)
# allHourlyData95.round(2).to_csv("../Regression_Scraper_Output/msp_hourly95_data.csv", header=True, index=False)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## ---

## Combine v9.2 CSV Files in Directory

In [49]:
# Create variable for files in directory
files = [f for f in os.listdir("data/92/msp/") if f.endswith(".csv")]

# files

In [50]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data92 = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/92/msp/" + file)
    all_data92 = pd.concat([all_data92, df])
    
all_data92

# 429240  rows, 214 columns

,Scenario,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Exterior Horizontal Sky Illuminance [lux](Hourly),Environment:Site Exterior Horizontal Beam Illuminance [lux](Hourly),Environment:Site Exterior Beam Normal Illuminance [lux](Hourly),Environment:Site Sky Diffuse Solar Radiation Luminous Efficacy [lum/W](Hourly),Environment:Site Beam Solar Radiation Luminous Efficacy [lum/W](Hourly),...,WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),LIVING_UNIT1:Zone Operative Temperature [C](Hourly),ATTIC_UNIT1:Zone Operative Temperature [C](Hourly),SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly)
0,1,01/01 01:00:00,-6.6500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.473385,-2.452456,378224.3700,24412822.31,0.0
1,1,01/01 02:00:00,-7.9875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.442514,-4.927244,424951.5244,27428867.30,0.0
2,1,01/01 03:00:00,-7.4250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.423659,-5.761719,436679.9847,28185891.01,0.0
3,1,01/01 04:00:00,-6.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.406824,-5.776434,436179.0562,28153558.15,0.0
4,1,01/01 05:00:00,-6.3250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.393755,-5.509733,446365.0397,28811021.35,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,49,12/31 20:00:00,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.213423,0.941696,368824.7907,23806118.26,0.0
8756,49,12/31 21:00:00,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.211417,0.902397,368662.3565,23795633.80,0.0
8757,49,12/31 22:00:00,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.195354,0.866909,380711.0241,24573325.57,0.0
8758,49,12/31 23:00:00,-0.3750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0,0.0,0,20.163436,0.745452,396038.6187,25562658.55,0.0


In [51]:
# Get wanted columns
all_data92_1 = all_data92[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

all_data92_1.head()

,Scenario,Date/Time,LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly),SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly)
0,1,01/01 01:00:00,0.0,378224.3700,24412822.31,0.0
1,1,01/01 02:00:00,0.0,424951.5244,27428867.30,0.0
2,1,01/01 03:00:00,0.0,436679.9847,28185891.01,0.0
3,1,01/01 04:00:00,0.0,436179.0562,28153558.15,0.0
4,1,01/01 05:00:00,0.0,446365.0397,28811021.35,0.0


In [52]:
# all_data92_1 = all_data92_1.dropna().reset_index(drop=True)

# all_data92_1

# #438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [53]:
# Rename Columns
all_data92_2 = all_data92_1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data92_2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [54]:
# Copy dataframe to modify and leave original df intact
get_annual92 = all_data92_2.copy(deep=True)

# get_annual92.head()

In [55]:
# Convert Joules to KBtu in dataframe
get_annual92["FanEnergy(kBtu)"] = get_annual92["FanEnergy[J](Hourly)"] * .00000094781712
get_annual92["HeatingEnergy(kBtu)"] = get_annual92["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual92["CoolingEnergy(kBtu)"] = get_annual92["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual92

In [56]:
# Get the annual energy per square foot
# Square foot is defined in the size variable

get_annual92["FanEnergy(kBtu/sf)"] = get_annual92["FanEnergy(kBtu)"] / size
get_annual92["HeatingEnergy(kBtu/sf)"] = get_annual92["HeatingEnergy(kBtu)"] / size
get_annual92["CoolingEnergy(kBtu/sf)"] = get_annual92["CoolingEnergy(kBtu)"] / size

In [57]:
# Drop columns
get_annual92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# get_annual92

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [58]:
# Get annual costs
get_annual92_costs = all_data92_2.copy(deep=True)

In [59]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual92_costs["FanCost($)"] = (get_annual92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual92_costs["HeatingCost($)"] = ((get_annual92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual92_costs["CoolingCost($)"] = (get_annual92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual92_costs

In [60]:
# Get the annual energy per square foot
# Square foot is defined in the size variable

get_annual92_costs["FanCost($/sf)"] = get_annual92_costs["FanCost($)"] / size
get_annual92_costs["HeatingCost($/sf)"] = get_annual92_costs["HeatingCost($)"] / size
get_annual92_costs["CoolingCost($/sf)"] = get_annual92_costs["CoolingCost($)"] / size

In [61]:
# Drop columns
get_annual92_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual92_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [62]:
# Combine data for energy and cost
allAnnualData92 = pd.merge(get_annual92, get_annual92_costs)

#allAnnualData2

In [63]:
# Add energy and cost totals
totalEnergy92 = allAnnualData92["FanEnergy(kBtu)"] + allAnnualData92["HeatingEnergy(kBtu)"] + allAnnualData92["CoolingEnergy(kBtu)"]
allAnnualData92["AnnualEnergy(kBtu)"] = totalEnergy92

totalCost92 = allAnnualData92["FanCost($)"] + allAnnualData92["HeatingCost($)"] + allAnnualData92["CoolingCost($)"]
allAnnualData92["AnnualCost($)"] = totalCost92


totalEnergyEUI92 = allAnnualData92["FanEnergy(kBtu/sf)"] + allAnnualData92["HeatingEnergy(kBtu/sf)"] + allAnnualData92["CoolingEnergy(kBtu/sf)"]
allAnnualData92["AnnualEnergy(kBtu/sf)"] = totalEnergyEUI92

totalCostEUI92 = allAnnualData92["FanCost($/sf)"] + allAnnualData92["HeatingCost($/sf)"] + allAnnualData92["CoolingCost($/sf)"]
allAnnualData92["AnnualCost($/sf)"] = totalCostEUI92


# allAnnualData92.head()

In [64]:
#list(allAnnualData92.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [65]:
# Group by scenario
annualData92 = allAnnualData92.groupby(["Scenario"], as_index=True)

#annualData92.sum().round(2)

In [66]:
annualDataFinal92 = annualData92.sum().round(2)

annualDataFinal92.head()

,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy(kBtu/sf),HeatingEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),FanCost($),HeatingCost($),CoolingCost($),FanCost($/sf),HeatingCost($/sf),CoolingCost($/sf),AnnualEnergy(kBtu),AnnualCost($),AnnualEnergy(kBtu/sf),AnnualCost($/sf)
Scenario,,,,,,,,,,,,,,,,,
1,17991384.19,1939.37,99954.87,4206.18,0.81,41.65,1.75,74.68,819.63,161.98,0.03,0.34,0.07,106100.42,1056.29,44.21,0.44
2,17991384.19,1978.10,96407.03,5184.91,0.82,40.17,2.16,76.18,790.54,199.67,0.03,0.33,0.08,103570.04,1066.38,43.15,0.44
3,17991384.19,2029.72,93130.47,6243.85,0.85,38.80,2.60,78.16,763.67,240.45,0.03,0.32,0.10,101404.03,1082.28,42.25,0.45
4,18570390.88,2056.93,91131.47,6839.73,0.86,37.97,2.85,79.21,747.28,263.39,0.03,0.31,0.11,100028.12,1089.88,41.68,0.45
5,19149772.61,2111.33,88496.31,7811.19,0.88,36.87,3.25,81.31,725.67,300.81,0.03,0.30,0.13,98418.83,1107.78,41.01,0.46


In [67]:
# Export grouped file to csv
annualDataFinal92.to_csv("../Regression_Scraper_Output/msp_annual92_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh for costs only, convert to kbtu for data<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [68]:
# # Copy dataframe to modify and leave original df intact
# get_monthly92 = all_data92_2.copy(deep=True)

# get_monthly92

In [69]:
# # Convert Joules to kWh and therms in new dataframe
# get_monthly92["FanEnergy(kBtu)"] = get_monthly92["FanEnergy[J](Hourly)"] * .00000094781712
# get_monthly92["HeatingEnergy(kBtu)"] = get_monthly92["HeatingEnergy[J](Hourly)"] * .00000094781712
# get_monthly92["CoolingEnergy(kBtu)"] = get_monthly92["CoolingEnergy[J](Hourly) "] * .00000094781712

# #get_monthly92

In [70]:
# # Drop columns
# get_monthly92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_monthly92

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [71]:
# # Copy dataframe to modify and leave original df intact
# get_monthly92_costs = all_data92_2.copy(deep=True)

# #get_monthly92

In [72]:
# # Convert Joules to KBtu in dataframe and add cost multiplier
# # Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# # Elec kbtu => Kwh (kbtu * .000000277777778)
# get_monthly92_costs["FanCost($)"] = (get_monthly92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
# get_monthly92_costs["HeatingCost($)"] = ((get_monthly92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
# get_monthly92_costs["CoolingCost($)"] = (get_monthly92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

# #get_monthly92_costs

In [73]:
# # Drop columns
# get_monthly92_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_monthly92_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [74]:
# # Combine data for energy and cost
# allMonthlyData92 = pd.merge(get_monthly92, get_monthly92_costs, how="inner")

# #allMonthlyData92

In [75]:
# # Add energy and cost totals

# totalEnergyMonthly92 = allMonthlyData92["FanEnergy(kBtu)"] + allMonthlyData92["HeatingEnergy(kBtu)"] + allMonthlyData92["CoolingEnergy(kBtu)"]
# totalCostMonthly92 = allMonthlyData92["FanCost($)"] + allMonthlyData92["HeatingCost($)"] + allMonthlyData92["CoolingCost($)"]
# allMonthlyData92["MonthlyEnergy(kBtu)"] = totalEnergyMonthly92
# allMonthlyData92["MonthlyCost($)"] = totalCostMonthly92

# #allMonthlyData92

In [76]:
# allMonthlyData92["Month"] = allMonthlyData92.Date_Time.str.slice(0,3)

# allMonthlyData92

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [77]:
# # Group by scenario and date/time for export to graphing
# MonthlyData92_2 = allMonthlyData92.groupby(["Scenario", "Month"], as_index=True)

# #MonthlyData92_2.sum()

In [78]:
# MonthlyDataFinal92 = MonthlyData92_2.sum()

# #MonthlyDataFinal92

In [79]:
# # Drop Illuminance column...don't need the totals
# MonthlyDataFinal92.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [80]:
# # Export grouped file to csv (used before grouped by month)
# MonthlyDataFinal92.round(2).to_csv("../Regression_Scraper_Output/msp_monthly92_data.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [81]:
# # Copy dataframe to modify and leave original df intact
# all_data92_3 = all_data92_1.copy(deep=True)

# #all_data92_3

In [82]:
# # Get wanted columns
# # Include hourly illumance
# all_data92_3 = all_data92_3[["Scenario",
#                "Date/Time",
#               "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
#               "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
#               "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
#               "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]
# #all_data92_3

In [83]:
# # Rename Columns
# get_hourly92 = all_data92_3.rename(columns={"Scenario":"Scenario",
#                               "Date/Time":"Date_Time",
#                              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
#                              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
#                              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
#                              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
#                              })

# #get_hourly92

In [84]:
# # Convert Joules to kBtus in new dataframe
# get_hourly92["FanEnergy(kBtu)"] = get_hourly92["FanEnergy[J](Hourly)"] * .00000094781712
# get_hourly92["HeatingEnergy(kBtu)"] = get_hourly92["HeatingEnergy[J](Hourly)"] * .00000094781712
# get_hourly92["CoolingEnergy(kBtu)"] = get_hourly92["CoolingEnergy[J](Hourly) "] * .00000094781712

# #get_hourly92

In [85]:
# # Drop columns
# get_hourly92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_hourly92

In [86]:
#list(get_hourly92.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [87]:
# # Copy dataframe to modify and leave original df intact
# get_hourly92_costs = all_data92_3.copy(deep=True)

# #get_hourly92_costs

In [88]:
# # Rename Columns
# get_hourly92_costs = get_hourly92_costs.rename(columns={"Scenario":"Scenario",
#                               "Date/Time":"Date_Time",
#                              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
#                              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
#                              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
#                              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
#                              })

# # get_hourly92_costs

In [89]:
# # Convert Joules to KBtu in dataframe and add cost multiplier
# # Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# # Elec kbtu => Kwh (kbtu * .000000277777778)
# get_hourly92_costs["FanCost($)"] = (get_hourly92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
# get_hourly92_costs["HeatingCost($)"] = ((get_hourly92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
# get_hourly92_costs["CoolingCost($)"] = (get_hourly92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

# ##get_hourly92_costs

In [90]:
# # Drop columns
# get_hourly92_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

# #get_hourly92_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [91]:
# # Combine data for energy and cost
# allHourlyData92 = pd.merge(get_hourly92, get_hourly92_costs, how="inner")

# #allHourlyData92

In [92]:
# #Extract hour, day, month.
# allHourlyData92["Month"] = allHourlyData92.Date_Time.str.slice(0,3)
# allHourlyData92["Day"] = allHourlyData92.Date_Time.str.slice(4,6)
# allHourlyData92["Hour"] = allHourlyData92["Date_Time"].astype(str).str[8:16]

# allHourlyData92

In [93]:
# # Add energy and cost totals

# totalEnergyhourly92 = allHourlyData92["FanEnergy(kBtu)"] + allHourlyData92["HeatingEnergy(kBtu)"] + allHourlyData92["CoolingEnergy(kBtu)"]
# totalCosthourly92 = allHourlyData92["FanCost($)"] + allHourlyData92["HeatingCost($)"] + allHourlyData92["CoolingCost($)"]
# allHourlyData92["HourlyEnergy(kBtu)"] = totalEnergyhourly92
# allHourlyData92["HourlyCost($)"] = totalCosthourly92

# #allHourlyData92

In [94]:
# # Export grouped file to csv (used before grouped by month)
# allHourlyData92.round(2).to_csv("../Regression_Scraper_Output/msp_hourly92_data.csv", header=True, index=False)